In [101]:
import gym
import numpy as np

In [102]:
env = gym.make('FrozenLake-v0')

## Value Iteration

In [103]:
def value_iteration(env, num_iterations=10000, gamma=0.9, threshold=1e-10):
    num_states = env.nS
    num_actions = env.nA

    values = np.zeros(num_states)

    for i in range(num_iterations):
        values_copy = np.copy(values)

        for state in range(num_states):
            Q_value = []

            for action in range(num_actions):
                next_state_rewards = []

                for transition in env.P[state][action]:
                    prob, next_state, reward, _ = transition
                    # Q_{t+1}(s, a) = p(s' | a,s) * (r + g * V_{t}(s'))
                    next_state_rewards.append((prob * (reward + gamma * values_copy[next_state])))
                
                Q_value.append(np.sum(next_state_rewards))
            
            values[state] = max(Q_value)
        
        if np.sum(np.fabs(values - values_copy)) < threshold:
            print(f'Value iteration converged at iteration {i+1}')
            break
    return values

def extract_policy(env, values, gamma=0.9):
    num_states = env.nS
    num_actions = env.nA

    policy = np.zeros(num_states, dtype=int)

    for state in range(num_states):
        Q_table = np.zeros(num_actions)
        for action in range(num_actions):
            for transition in env.P[state][action]:
                prob, next_state, reward, _ = transition
                Q_table[action] += prob * (reward + gamma * values[next_state])
        policy[state] = np.argmax(Q_table)
    return policy

In [111]:
V = value_iteration(env)
print(V)

Value iteration converged at iteration 159
[0.0688909  0.06141457 0.07440976 0.05580732 0.09185454 0.
 0.11220821 0.         0.14543635 0.24749695 0.29961759 0.
 0.         0.3799359  0.63902015 0.        ]


In [110]:
pi = extract_policy(env, V)
print(pi)

[0 3 0 3 0 0 0 0 3 1 0 0 0 2 1 0]


## Policy Iteration

In [106]:
def compute_values(env, policy, gamma=0.9, threshold=1e-10):
    num_states = env.nS
    num_actions = env.nA

    values = np.zeros(num_states)
    while True:
        values_copy = np.copy(values)
        for state in range(num_states):
            action = policy[state]
            value_sum = 0
            for prob, next_state, reward, _ in env.P[state][action]:
                value_sum += prob * (reward + gamma * values_copy[next_state])
            values[state] = value_sum
        if np.sum(np.fabs(values - values_copy)) < threshold:
            break
    return values
    
def policy_iteration(env, num_iterations=10000, gamma=0.9, threshold=1e-10):
    num_states = env.nS
    num_actions = env.nA

    policy = np.zeros(num_states, dtype=int)
    for i in range(num_iterations):
        values = compute_values(env, policy, gamma=gamma, threshold=threshold)
        new_policy = extract_policy(env, values, gamma=gamma)
        if (np.all(policy == new_policy)):
            print(f'Policy iteration converged at step {i+1}')
            break
        policy = new_policy
    return new_policy

In [112]:
pi_2 = policy_iteration(env)
print(pi_2)

Policy iteration converged at step 6
[0 3 0 3 0 0 0 0 3 1 0 0 0 2 1 0]


In [113]:
V_2 = compute_values(env, pi_2)
print(V_2)

[0.0688909  0.06141457 0.07440976 0.05580732 0.09185454 0.
 0.11220821 0.         0.14543635 0.24749695 0.29961759 0.
 0.         0.3799359  0.63902015 0.        ]


In [108]:
def eval(env, policy, values=None, max_episode_len=100):

    state = env.reset()

    i = 0
    while i < max_episode_len:
        env.render()

        action = policy[state]
        state, _, done, _ = env.step(action)

        if values is not None:
            print(action, values[state])
        
        if done:
            break
        i += 1

In [109]:
eval(env, pi_2, V_2)


SFFF
FHFH
FFFH
HFFG
0 0.06889090478451236
  (Left)
SFFF
FHFH
FFFH
HFFG
0 0.06889090478451236
  (Left)
SFFF
FHFH
FFFH
HFFG
0 0.06889090478451236
  (Left)
SFFF
FHFH
FFFH
HFFG
0 0.06889090478451236
  (Left)
SFFF
FHFH
FFFH
HFFG
0 0.06889090478451236
  (Left)
SFFF
FHFH
FFFH
HFFG
0 0.06889090478451236
  (Left)
SFFF
FHFH
FFFH
HFFG
0 0.09185453975699098
  (Left)
SFFF
FHFH
FFFH
HFFG
0 0.1454363546887556
  (Left)
SFFF
FHFH
FFFH
HFFG
3 0.24749695454938817
  (Up)
SFFF
FHFH
FFFH
HFFG
1 0.3799359011282286
  (Down)
SFFF
FHFH
FFFH
HFFG
2 0.6390201480990125
  (Right)
SFFF
FHFH
FFFH
HFFG
1 0.3799359011282286
  (Down)
SFFF
FHFH
FFFH
HFFG
2 0.3799359011282286
  (Right)
SFFF
FHFH
FFFH
HFFG
2 0.3799359011282286
  (Right)
SFFF
FHFH
FFFH
HFFG
2 0.24749695454938817
  (Right)
SFFF
FHFH
FFFH
HFFG
1 0.3799359011282286
  (Down)
SFFF
FHFH
FFFH
HFFG
2 0.3799359011282286
  (Right)
SFFF
FHFH
FFFH
HFFG
2 0.24749695454938817
  (Right)
SFFF
FHFH
FFFH
HFFG
1 0.3799359011282286
  (Down)
SFFF
FHFH
FFFH
HFFG
2 0.24749695454